In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# pb_user 테이블 읽기 = user_df
# consulting 테이블 읽기 = consult_df
# 카카오톡스토어 파마브로스 매출 데이터 테이블 읽기 = kakao_df1
# 카카오톡스토어 스타팜 매출 데이터테이블 읽기 = kakao_df2
# 친한스토어 매출 데이터테이블 읽기 = fs_df
# 데일리 엑세스 로그 테이블 읽기 = log_df

consult_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_consulting_export_2025-02-03_093017.csv')
user_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_pb_user_export_2025-02-03_092947.csv')
kakao_df1 = pd.read_excel('/Users/seanyu_mac/Downloads/20250203093147.xlsx')
kakao_df2 = pd.read_excel('/Users/seanyu_mac/Downloads/20250203094213.xlsx')
log_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_daily_access_log_export_2025-02-03_093157.csv')
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_2024-02-03~2025-02-03.xlsx')

# 레포팅 날짜 기준을 설정
start_day = datetime(2024, 12, 1)
mid_day = datetime(2025, 1, 1)
end_day = datetime(2025, 2, 1)

# rate 함수 정의
def rate(before, after):
    percent = (after - before) / before * 100
    percent = np.round(percent, 1)
    if percent > 0:
        result = f'{percent}% 증가'
        return result
    else:
        result = f'{percent}% 감소'
        return result

# kakao_df 머지하여 생성하고 필요한 컬럼 정리
# 배송상태에서 특이값 제거
kakao_df = pd.concat([kakao_df1, kakao_df2])
remove_txts = ['204 결제 취소 완료',
               '208 환불 완료',
               '507 반품 결제 취소 완료',
               '303 결제 취소 완료',
               '404 교환 반송 중',
               '505 반품 반송 완료']

kakao_df.drop(kakao_df[kakao_df['주문상태'].isin(remove_txts)].index, inplace=True)

# 필요한 컬럼만 남기고 영문 컬럼명으로 재정의 : 'payment_id', 'order_id', 'product_name', 'option_name', 'quantity', 'sales'
kakao_df = kakao_df[['결제번호', '주문번호', '주문일', '주문상태', '상품명', '옵션', '수량', '정산기준금액']]
pre_col = ['결제번호', '주문번호', '주문일', '주문상태', '상품명', '옵션', '수량', '정산기준금액']
post_col = ['payment_id', 'order_id', 'order_date', 'status', 'product_name', 'option_name', 'quantity', 'sales']
col_dict = dict(zip(pre_col, post_col))
kakao_df.rename(columns=col_dict, inplace=True)

# order_date를 datetime 값으로 변환
kakao_df['order_date'] = pd.to_datetime(kakao_df['order_date'])

# user_df를 필요 컬럼 남기고 정리 : 'id', 'email', 'name', 'last_login_dt', 'reg_dt', 'del_yn', 'gender', 'birthyear'
user_df = user_df[['id', 'email', 'name', 'last_login_dt', 'reg_dt', 'del_yn', 'gender', 'birthyear']]

# user_df 날짜 컬럼들 datetime 타입으로 재정의
user_df['last_login_dt'] = pd.to_datetime(user_df['last_login_dt'], format='mixed')
user_df['reg_dt'] = pd.to_datetime(user_df['reg_dt'], format='mixed')

# del_yn 값이 Y인 사용자 제거
user_df.drop(user_df[user_df['del_yn'] == 'Y'].index, inplace=True)

# consult_df 에서 필요한 컬럼만 재정의 : 'id', 'user_id', 'reg_dt', 'answer_dt', 'public_yn'
consult_df = consult_df[['id', 'user_id', 'reg_dt', 'answer_dt', 'public_yn']]

# consult_df 의 날짜 컬럼들을 datetime 값으로 재정의
consult_df['reg_dt'] = pd.to_datetime(consult_df['reg_dt'], format='mixed')
consult_df['answer_dt'] = pd.to_datetime(consult_df['answer_dt'], format='mixed')

# 지지난달 월과 지난달 월
pre_month = start_day.month
post_month = mid_day.month

# 총 회원가입자 수 : total_users_num
total_users_num = user_df.index.size
monthly_user_df = pd.DataFrame(user_df.groupby(pd.Grouper(key='reg_dt', freq='1M')).size().reset_index())
monthly_user_df = monthly_user_df[monthly_user_df['reg_dt'] >= datetime(2024, 1, 1)]
monthly_user_df['month'] = monthly_user_df['reg_dt'].dt.month
monthly_user_df['year'] = monthly_user_df['reg_dt'].dt.year

# 지지난달 회원가입자 수 : pre_month_users_num
# 지난달 회원가입자 수 : post_month_users_num
# 회원가입자 수 증감율 : month_users_rate
con1 = monthly_user_df['year'] == start_day.year
con2 = monthly_user_df['month'] == start_day.month
pre_month_users_num = monthly_user_df.loc[con1 & con2, 0]
pre_month_users_num = pre_month_users_num[pre_month_users_num.index[0]]
con1 = monthly_user_df['year'] == mid_day.year
con2 = monthly_user_df['month'] == mid_day.month
post_month_users_num = monthly_user_df.loc[con1 & con2, 0]
post_month_users_num = post_month_users_num[post_month_users_num.index[0]]
month_users_rate = rate(pre_month_users_num, post_month_users_num)

# 레포트 제출일 기준 총 누적 상담 건수 = total_consult_num
total_consult_num = consult_df.index.size

# 지지난달과 지난달 상담 인입 건수 = pre_consult_num, post_consult_num
# 2달간 상담 인입률 증감율 = consult_rate
con1 = consult_df['reg_dt'] >= start_day
con2 = consult_df['reg_dt'] < mid_day
pre_consult_num = consult_df.loc[con1 & con2].index.size
con1 = consult_df['reg_dt'] >= mid_day
con2 = consult_df['reg_dt'] < end_day
post_consult_num = consult_df.loc[con1 & con2].index.size
consult_rate = rate(pre_consult_num, post_consult_num)

# daily_access_log 테이블 분석하여 지지난달과 지난달 MAU 추출 = pre_month_mau, post_month_mau, mau_rate
log_df['access_date'] = pd.to_datetime(log_df['access_date'], format='mixed')
log_df['created_at'] = pd.to_datetime(log_df['created_at'], format='mixed')
log_df.drop(log_df[log_df['user_id'].duplicated() == True].index, inplace=True)
con1 = log_df['access_date'] >= start_day
con2 = log_df['access_date'] < mid_day
pre_month_mau = log_df[con1 & con2].index.size
con1 = log_df['access_date'] >= mid_day
con2 = log_df['access_date'] < end_day
post_month_mau = log_df[con1 & con2].index.size
mau_rate = rate(pre_month_mau, post_month_mau)

# fs_df 컬럼 정리하고 kakao_df 와 머지하여 sales_df 생성
con1 = fs_df['주문 상태'] == '배송완료'
con2 = fs_df['주문 상태'] == '배송중'
con3 = fs_df['주문 상태'] == '배송준비'
con4 = fs_df['주문 상태'] == '구매확정'
con5 = fs_df['주문 상태'] == '결제완료'
fs_df = fs_df[con1 | con2 | con3 | con4 | con5]

fs_df = fs_df[['결제 번호', '상품 번호', '결제 완료일', '주문 상태', '상품명', '옵션명', '수량', '정산가']]
col_dict = dict(zip(fs_df.columns.tolist(), kakao_df.columns.tolist()))
fs_df.rename(columns=col_dict, inplace=True)
sales_df = pd.concat([kakao_df, fs_df])

# 총 누적 판매 금액 = total_sales
# 일 평균 판매 금액 = daily_sales
# 주문당 평균 객단가 = arpu
# 지지난달 지난달 매출과 증감율 pre_sales, post_sales, sales_rate
total_sales = sales_df['sales'].sum()
sales_df['order_day'] = sales_df['order_date'].dt.date
daily_sales = np.round(total_sales / sales_df['order_day'].unique().size, 1)
arpu = np.round(total_sales / sales_df['order_id'].unique().size, 1)
con1 = sales_df['order_date'] >= start_day
con2 = sales_df['order_date'] < mid_day
pre_sales = sales_df.loc[con1 & con2, 'sales'].sum()
con1 = sales_df['order_date'] >= mid_day
con2 = sales_df['order_date'] < end_day
post_sales = sales_df.loc[con1 & con2, 'sales'].sum()
sales_rate = rate(pre_sales, post_sales)

print(f'1. 지지난달 {pre_month}월과 지난달 {post_month}월을 비교')
print(f'2. {pre_month}월 기준 총 회원 가입 유저 수는 {total_users_num:,}명')
print(f'3. {pre_month}월 회원 가입자 수: {pre_month_users_num:,}명 / {post_month}월 회원 가입자 수: {post_month_users_num:,}명으로 전월 대비 {month_users_rate}')
print(f'4. {post_month}월 기준, 총 누적 상담 건수는 {total_consult_num:,}건')
print(f'5. {pre_month}월 상담 수: {pre_consult_num:,}건, {post_month}월 상담 수: {post_consult_num:,}건으로 전월 대비 {consult_rate}')
print(f'6. {post_month}월 MAU {post_month_mau:,}명 (직전 {pre_month}월 {pre_month_mau:,}명 대비 {mau_rate})')
print()
print(f'1. 누적 판매금액 (최근 1년) : {total_sales:,}원 (2024년 1월 11일 오픈)')
print(f'2. 일 평균 판매액 : {daily_sales:,}원')
print(f'3. 객단가 : {arpu}원')
print(f'4. {pre_month}월 매출 : {pre_sales:,}원 / {post_month}월 매출 : {post_sales:,}원 ({sales_rate})')

/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_1340/1671600740.py:12: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  consult_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_consulting_export_2025-02-03_093017.csv')
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1. 지지난달 12월과 지난달 1월을 비교
2. 12월 기준 총 회원 가입 유저 수는 29,116명
3. 12월 회원 가입자 수: 1,437명 / 1월 회원 가입자 수: 1,872명으로 전월 대비 30.3% 증가
4. 1월 기준, 총 누적 상담 건수는 38,232건
5. 12월 상담 수: 1,920건, 1월 상담 수: 2,045건으로 전월 대비 6.5% 증가
6. 1월 MAU 1,124명 (직전 12월 906명 대비 24.1% 증가)

1. 총 누적 판매금액 : 221,526,330원 (2024년 1월 11일 오픈)
2. 일 평균 판매액 : 608,588.8원
3. 객단가 : 47507.3원
4. 12월 매출 : 22,719,670원 / 1월 매출 : 23,973,240원 (5.5% 증가)


/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_1340/1671600740.py:80: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_user_df = pd.DataFrame(user_df.groupby(pd.Grouper(key='reg_dt', freq='1M')).size().reset_index())


In [17]:
# 월별 회원가입자 수 추이
user_df['sign_up_date'] = user_df['reg_dt'].dt.date
user_df['sign_up_date'] = pd.to_datetime(user_df['sign_up_date'])
temp_df = user_df[user_df['reg_dt'] >= datetime(2024, 1, 1)]
temp_df['sign_up_month'] = temp_df['reg_dt'].dt.month
temp_df['sign_up_year'] = temp_df['reg_dt'].dt.year
temp_df['sign_up_year_month'] = temp_df['sign_up_date'].dt.to_period('M')
temp_df = pd.DataFrame(temp_df.groupby('sign_up_year_month').size())
temp_df

/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_1340/3394042185.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['sign_up_month'] = temp_df['reg_dt'].dt.month
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_1340/3394042185.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['sign_up_year'] = temp_df['reg_dt'].dt.year
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_1340/3394042185.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

,0
sign_up_year_month,
2024-01,2380
2024-02,1279
2024-03,2259
2024-04,1815
2024-05,1978
2024-06,1960
2024-07,1988
2024-08,1382
2024-09,1139


In [19]:
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
sales_df['year_month'] = sales_df['order_date'].dt.to_period('M')
temp_df = pd.DataFrame(sales_df.groupby('year_month')['sales'].sum())
temp_df

,sales
year_month,
2024-02,10922880
2024-03,17972710
2024-04,22886150
2024-05,17269740
2024-06,14477520
2024-07,16126050
2024-08,19530760
2024-09,17341110
2024-10,14629200


In [21]:
txts = kakao_df['status'].unique().tolist()
print(f'정제한 상품 데이터의 상태는 {txts}\n제거한 상품 데이터의 상태는 {remove_txts}')

정제한 상품 데이터의 상태는 ['202 배송 요청', '305 배송 완료', '304 배송 중', '601 구매 결정']
제거한 상품 데이터의 상태는 ['204 결제 취소 완료', '208 환불 완료', '507 반품 결제 취소 완료', '303 결제 취소 완료', '404 교환 반송 중', '505 반품 반송 완료']
